Blast markers versus dipterian genomes: Anopheles gambiae, Aedes aegyptii, Culex pipiens, Drosophila melanogaster.

Results:
- 591 of 663 markers with unique location in agam genome left for analysis

Minor results:
- add unaligned species to metadata
- reorder and rename columns

## Libraries and parameters

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pybedtools
from Bio import AlignIO
from Bio.Blast.Applications import NcbiblastnCommandline


In [ ]:
DATA_DIR = "../../../data/"
# blast db from Anopheles gambiae, Aedes segypti, Culex pipiens, Drosophila melanogaster genomes
COMB_DB = DATA_DIR + 'genome/AgAaCpDm' 
# out
AMPL_FILE = "data/20180619_comb_ampl.maf"
META_FILE = "data/20180619_comb_ampl_data.csv"
BLAST_FILE = "data/20180706_blast_4sp.xml"
UPD_META_FILE = "data/20180706_comb_ampl_data.xlsx"
UPD_AMPL_FILE = "data/20180706_comb_ampl.maf"

## BLAST run

In [3]:
# extract agam sequences for markers removing alignment gaps
# headers are 0-based numbers of markers
agam_fa = 'data/temp_ampl.fasta'

with open('data/temp_ampl.fasta', 'w') as o:
    i = 0
    ampl_seq = AlignIO.parse(AMPL_FILE, "maf")
    for a in ampl_seq:
        o.write(">{}\n".format(i))
        o.write(str(a[0].seq).replace('-',''))
        o.write('\n')
        i += 1
    
!head {agam_fa}

>0
CGTAACGGTAGAGGACGTTTCGTTGGTGTACCTGGTAGCGATAGAAAACCACTGGTACTATCAACATATCTCGATGCACAAGAACACCTGCCATACGCTGATGATTCCAACGCGGTAACGCCGATGTCGGAAGAAAATGGTGCAATCATCGTTCCAGTATACTATGCTAATTTAGGTACACTAT
>1
AGTGCTCCATTTTCGTTATTAAACACTAGCCTTCTTTGTCTGGAGTCTAACATTAAAGGTGGAATCACAGCATATGAGCGCATTTTCTTATCCTCCATGGCCTAAAAACATCGGAAGAATCGTATGAATAAACATTTTCATGGTTACCATTTAATACTATAAGATCCATTTGTTAACTTACCTGTTCTTGAAGACCATCCATGATCTCTTCAAGATCGGCTTCAGATTTTATTCTAGAACCTACACGATTAAATCGTTCCTTATCTTCCCGCTGCTTTCGAAGCTCC
>2
GCACACTGTGTCGTTAGCTTTAACCTTGAAACTTCTACCCTCCAGCAAGCGAGTGAGAGCGCCGCGAGAGTTGTGCCAGTACGCGGCTCCACCATGCGCCCGTTTCGGTCAGATCGTCGAAATCGCTTCCTAAAT
>3
GAGGATATAACGGCGAAAGTTTGGAGCATAAAGTCGGCGGATCAGTTGCAGTGCTTCCTACGACATATTGTGAAGGCGTTCACGGACAGCTATCCCCACGCGCGAATACCGGAGCGTTGGGCTCAGACTGCACTGCAGCTGGGGCTTAGCTGCTCGTCGCGCCACTATGCCGGGCGCAGTTTGCAAGTGTTCCGCTCGCTTAATGTACCAATCAATTCGCGCATGCTGT
>4



In [4]:
help(NcbiblastnCommandline)

Help on class NcbiblastnCommandline in module Bio.Blast.Applications:

class NcbiblastnCommandline(_NcbiblastMain2SeqCommandline)
 |  Wrapper for the NCBI BLAST+ program blastn (for nucleotides).
 |  
 |  With the release of BLAST+ (BLAST rewritten in C++ instead of C), the NCBI
 |  replaced the old blastall tool with separate tools for each of the searches.
 |  This wrapper therefore replaces BlastallCommandline with option -p blastn.
 |  
 |  For example, to run a search against the "nt" nucleotide database using the
 |  FASTA nucleotide file "m_code.fasta" as the query, with an expectation value
 |  cut off of 0.001, saving the output to a file in XML format:
 |  
 |  >>> from Bio.Blast.Applications import NcbiblastnCommandline
 |  >>> cline = NcbiblastnCommandline(query="m_cold.fasta", db="nt", strand="plus",
 |  ...                               evalue=0.001, out="m_cold.xml", outfmt=5)
 |  >>> cline
 |  NcbiblastnCommandline(cmd='blastn', out='m_cold.xml', outfmt=5, query='m_cold

In [5]:
# generate blast command
cline = NcbiblastnCommandline(cmd='blastn', out=BLAST_FILE, outfmt=5, query=agam_fa, 
                              db=COMB_DB, evalue=0.001)
print(cline)

blastn -out data/20180706_blast_4sp.xml -outfmt 5 -query data/temp_ampl.fasta -db /Users/am60/data/genome/AgAaCpDm -evalue 0.001


In [6]:
# run blast command. Return values are stdout and stderr
cline()

('', '')

## BLAST parse

In [7]:
# parse hits with NCBIXML
# http://biopython.org/DIST/docs/tutorial/Tutorial.html#fig:blastrecord
from Bio.Blast import NCBIXML
blast_records = list(NCBIXML.parse(open(BLAST_FILE)))
# http://biopython.org/DIST/docs/api/Bio.Blast.Record.HSP-class.html
test_hsp = blast_records[0].alignments[1].hsps[0] 
print(str(test_hsp))
print(test_hsp.identities, test_hsp.align_length)

Score 151 (279 bits), expectation 1.9e-73, alignment length 184
Query:       1 CGTAACGGTAGAGGACGTTTCGTTGGTGTACCTGGTAGCGATAGA...TAT 184
               ||||||||||||||||||||||| || ||||||||||||||||||...|||
Sbjct:  126707 CGTAACGGTAGAGGACGTTTCGTGGGCGTACCTGGTAGCGATAGA...TAT 126524
173 184


In [19]:
# extract nececcary blast data
from collections import defaultdict
bd = dict()
species = ('AgamP3', 'CpipJ2', 'AaegL5', 'BDGP6')
for record in blast_records:
    q = record.query
    bd[q] = defaultdict(list)
    for aln in record.alignments:
        #extract species from hit definition
        (s,c) = [l.split(':')[1:3] for l in aln.hit_def.split(' ') if len(l.split(':')) > 2][0]
        if s not in species:
            raise ValueError('Unknown genome ' + s)
#         if s not in bd[q].keys():
#             bd[q][s] = defaultdict(list)
        for hsp in aln.hsps:
            if (hsp.num_alignments is not None):
                raise ValueError('Many alignmed fragments per HSP:\n' + str(hsp))
            bd[q][s + '_length'].append(hsp.align_length)
            bd[q][s + '_identity'].append(hsp.identities / hsp.align_length)
            bd[q][s + '_q_span'].append('{}-{}'.format(hsp.query_start, hsp.query_end))
            bd[q][s + '_s_span'].append('{}:{}-{}'.format(c, hsp.sbjct_start, hsp.sbjct_end))
            bd[q][s + '_expect'].append(hsp.expect)
            i+=1
        bd[q][s + '_hits'] = len(bd[q][s + '_length'])
    for (k,v) in bd[q].items():
        if type(bd[q][k]) is list:
            bd[q][k] = ';'.join([str(s) for s in v])
            
bd['129']
# example of duplicated marker

defaultdict(list,
            {'AaegL5_expect': '5.79208e-131;1.62185e-126',
             'AaegL5_hits': 2,
             'AaegL5_identity': '0.9196428571428571;0.9088235294117647',
             'AaegL5_length': '336;340',
             'AaegL5_q_span': '8-343;4-343',
             'AaegL5_s_span': '3:25284012-25283677;2:45578367-45578028',
             'AgamP3_expect': '6.98078e-180;3.43696e-138',
             'AgamP3_hits': 2,
             'AgamP3_identity': '1.0;0.9294117647058824',
             'AgamP3_length': '343;340',
             'AgamP3_q_span': '1-343;4-343',
             'AgamP3_s_span': '2L:46338834-46339176;2R:39348407-39348746',
             'BDGP6_expect': '1.65664e-111;1.00412e-103',
             'BDGP6_hits': 2,
             'BDGP6_identity': '0.8808139534883721;0.8716417910447761',
             'BDGP6_length': '344;335',
             'BDGP6_q_span': '1-343;8-342',
             'BDGP6_s_span': '2R:11893567-11893910;3R:31740347-31740681',
             'CpipJ2_expect': '2.

In [26]:
# transform keys to int
bd =  {int(k): v for k, v in bd.items()}
bd[1]

defaultdict(list,
            {'AaegL5_expect': '8.8269e-59',
             'AaegL5_hits': 1,
             'AaegL5_identity': '0.8268551236749117',
             'AaegL5_length': '283',
             'AaegL5_q_span': '4-286',
             'AaegL5_s_span': '3:210133184-210132924',
             'AgamP3_expect': '7.77149e-149',
             'AgamP3_hits': 1,
             'AgamP3_identity': '1.0',
             'AgamP3_length': '287',
             'AgamP3_q_span': '1-287',
             'AgamP3_s_span': '2L:2926449-2926735',
             'CpipJ2_expect': '2.43691e-64',
             'CpipJ2_hits': 1,
             'CpipJ2_identity': '0.8333333333333334',
             'CpipJ2_length': '288',
             'CpipJ2_q_span': '1-286',
             'CpipJ2_s_span': 'supercont3.35:120152-120424'})

## Combine metadata

In [61]:
# load metadata
init_meta = pd.read_csv(META_FILE, index_col=0)
init_meta

,aligned_len,aligned_species,chr,end,id_lineages,indels,informativeness,seqid,snvs,start,...,target_snvs,target_start,unid_species,exon,gene,intron,mRNA,repeat,utr,id lineage proportion
0,185,21,2L,2403278,14,5,0.666667,AgamP3.chr2L,31,2403094,...,21,2403145,"[{'AmelC1', 'AgamP3', 'AaraD1', 'AgamM1', 'Aqu...",E013980A;E013980B;E013980C,AGAP004707,None,AGAP004707-RA;AGAP004707-RB;AGAP004707-RC,None,None,0.666667
1,345,21,2L,2926735,18,96,0.857143,AgamP3.chr2L,65,2926448,...,52,2926537,"[{'AgamM1', 'AaraD1', 'AgamP3'}, {'AsteI2', 'A...",E014129A;E014130A,AGAP004734,Yes,AGAP004734-RA,None,None,0.857143
2,199,17,2L,3249178,13,108,0.764706,AgamP3.chr2L,19,3249043,...,13,3249077,"[{'AmerM1', 'AgamP3', 'AaraD1', 'AgamM1', 'Aga...",None,None,None,None,None,None,0.764706
3,234,21,2L,5026687,15,5,0.714286,AgamP3.chr2L,43,5026458,...,35,5026511,"[{'AmelC1', 'AmerM1', 'AgamP3', 'AgamM1', 'Aqu...",E014976A,AGAP004892,None,AGAP004892-RA,None,None,0.714286
4,321,21,2L,5861628,15,191,0.714286,AgamP3.chr2L,10,5861468,...,8,5861515,"[{'AmelC1', 'AgamP3', 'AaraD1', 'AgamM1', 'Aga...",None,None,None,None,None,None,0.714286
5,156,21,2L,6015616,13,5,0.619048,AgamP3.chr2L,25,6015462,...,11,6015528,"[{'AmelC1', 'AmerM1', 'AgamP3', 'AaraD1', 'Aga...",E015082A,AGAP004919,None,AGAP004919-RA,None,Yes,0.619048
6,239,21,2L,6045216,14,103,0.666667,AgamP3.chr2L,28,6045074,...,19,6045125,"[{'AdirW1', 'AmerM1', 'AgamP3', 'AaraD1', 'Aga...",None,None,None,None,None,None,0.666667
7,272,20,2L,6091469,14,7,0.700000,AgamP3.chr2L,34,6091202,...,17,6091291,"[{'AmelC1', 'AgamP3', 'AaraD1', 'AquaS1', 'Aga...",E015094A,AGAP004923,None,AGAP004923-RA,None,None,0.700000
8,163,19,2L,6492034,13,40,0.684211,AgamP3.chr2L,29,6491898,...,23,6491957,"[{'AmelC1', 'AmerM1', 'AgamP3', 'AaraD1', 'Aga...",None,None,None,None,None,None,0.684211
9,228,21,2L,6534328,10,0,0.476190,AgamP3.chr2L,32,6534100,...,21,6534177,"[{'AfunF1', 'AdirW1', 'AmerM1', 'AgamP3', 'Ast...",E015172A;E015175B,AGAP004943,None,AGAP004943-RA;AGAP004943-RB,None,None,0.476190


In [62]:
# combine metadata
comb_meta = pd.concat([init_meta, pd.DataFrame(bd).T], axis=1)
comb_meta.shape

(663, 47)

In [63]:
comb_meta.columns

Index(['aligned_len', 'aligned_species', 'chr', 'end', 'id_lineages', 'indels',
       'informativeness', 'seqid', 'snvs', 'start', 'target_aligned_len',
       'target_end', 'target_indels', 'target_snvs', 'target_start',
       'unid_species', 'exon', 'gene', 'intron', 'mRNA', 'repeat', 'utr',
       'id lineage proportion', 'AaegL5_expect', 'AaegL5_hits',
       'AaegL5_identity', 'AaegL5_length', 'AaegL5_q_span', 'AaegL5_s_span',
       'AgamP3_expect', 'AgamP3_hits', 'AgamP3_identity', 'AgamP3_length',
       'AgamP3_q_span', 'AgamP3_s_span', 'BDGP6_expect', 'BDGP6_hits',
       'BDGP6_identity', 'BDGP6_length', 'BDGP6_q_span', 'BDGP6_s_span',
       'CpipJ2_expect', 'CpipJ2_hits', 'CpipJ2_identity', 'CpipJ2_length',
       'CpipJ2_q_span', 'CpipJ2_s_span'],
      dtype='object')

## Add forgotten data from alignment - unaln_species

In [64]:
# list of all species
all_sp = []
for aln in AlignIO.parse(AMPL_FILE, "maf"):
    if len(aln) == 21:
        all_sp = [seq.id.split('.')[0] for seq in aln]
    break
all_sp

['AgamP3',
 'AgamS1',
 'AgamM1',
 'AmerM1',
 'AaraD1',
 'AquaS1',
 'AmelC1',
 'AchrA1',
 'AepiE1',
 'AminM1',
 'AculA1',
 'AfunF1',
 'AsteS1',
 'AsteI2',
 'AmacM1',
 'AfarF1',
 'AdirW1',
 'AsinS1',
 'AatrE1',
 'AdarC2',
 'AalbS1']

In [65]:
# unaligned species per amplicon
unaln = dict()
i = 0
for aln in AlignIO.parse(AMPL_FILE, "maf"):
    aln_sp = [seq.id.split('.')[0] for seq in aln]
    unaln[i] = ';'.join([sp for sp in all_sp if sp not in aln_sp])
    i+=1
unaln[662]

'AfunF1;AmacM1;AfarF1;AdirW1;AsinS1;AatrE1;AdarC2;AalbS1'

In [66]:
# add to metadata
comb_meta['unaligned_species'] = pd.Series(unaln)
comb_meta.head()

,aligned_len,aligned_species,chr,end,id_lineages,indels,informativeness,seqid,snvs,start,...,BDGP6_length,BDGP6_q_span,BDGP6_s_span,CpipJ2_expect,CpipJ2_hits,CpipJ2_identity,CpipJ2_length,CpipJ2_q_span,CpipJ2_s_span,unaligned_species
0,185,21,2L,2403278,14,5,0.666667,AgamP3.chr2L,31,2403094,...,NaN,NaN,NaN,1.87203e-73,1,0.9402173913043478,184,1-184,supercont3.1170:126707-126524,
1,345,21,2L,2926735,18,96,0.857143,AgamP3.chr2L,65,2926448,...,NaN,NaN,NaN,2.43691e-64,1,0.8333333333333334,288,1-286,supercont3.35:120152-120424,
2,199,17,2L,3249178,13,108,0.764706,AgamP3.chr2L,19,3249043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AsinS1;AatrE1;AdarC2;AalbS1
3,234,21,2L,5026687,15,5,0.714286,AgamP3.chr2L,43,5026458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,321,21,2L,5861628,15,191,0.714286,AgamP3.chr2L,10,5861468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


## Remove markers duplicated in agam

In [74]:
# 72 markers were removed as duplicates in agam
comb_meta = comb_meta[comb_meta['AgamP3_hits'] == 1]
comb_meta.shape

(591, 48)

In [75]:
# write filtered alignment 
i = 0
with open(UPD_AMPL_FILE, "w") as o:
    for aln in AlignIO.parse(AMPL_FILE, "maf"):
        if i in comb_meta.index:
            AlignIO.write(aln, o, "maf")
        i += 1
! grep -c 'AgamP3' {UPD_AMPL_FILE}

591


In [76]:
# reset index for compatibility with filtered alignment
comb_meta.reset_index(drop=True, inplace=True)
comb_meta.index

RangeIndex(start=0, stop=591, step=1)

## Reorder columns and save

In [77]:
# reorder and rename columns
comb_meta = comb_meta[['chr', 'start', 'end', 'seqid', # agam coordinates
            'aligned_species', 'unaligned_species', 'aligned_len', 'indels', 'snvs', # amplicon stats
            'target_start', 'target_end', 'target_aligned_len', 'target_indels', 'target_snvs', # target stats
            'id_lineages', 'informativeness', 'unid_species', # identification power
            'exon', 'gene', 'intron', 'mRNA', 'repeat', 'utr', # agam annotations
            'AgamP3_hits', 'AgamP3_length', 'AgamP3_identity', 
            'AgamP3_q_span', 'AgamP3_s_span', 'AgamP3_expect', # agam blast
            'AaegL5_hits', 'AaegL5_length', 'AaegL5_identity',
            'AaegL5_q_span', 'AaegL5_s_span', 'AaegL5_expect', # aedes blast
            'CpipJ2_hits', 'CpipJ2_length', 'CpipJ2_identity', 
            'CpipJ2_q_span', 'CpipJ2_s_span', 'CpipJ2_expect', # culex blast
            'BDGP6_hits', 'BDGP6_length', 'BDGP6_identity', 
            'BDGP6_q_span', 'BDGP6_s_span','BDGP6_expect' # drosophila blast
            #'id lineage proportion' # duplicate to informativeness, remove
            ]]
comb_meta = comb_meta.rename(columns={"aligned_species": "num_aligned_species", 
                              "indels": "total_indels",
                              "snvs": "total_snvs",
                              "id_lineages":"num_id_lineages",
                              "informativeness": "prop_id_lineages"
                             })
comb_meta.shape

(591, 47)

In [79]:
comb_meta.to_excel(UPD_META_FILE)

## Sandbox

In [10]:
# parse hits with SearchIO - it does not extract identities/matches
from Bio import SearchIO
blast_qresult = list(SearchIO.parse(BLAST_FILE, "blast-xml"))
for hit in blast_qresult[129]: 
    for hsp in hit:
        print(hsp)
#hierarchy results->hit->hsp->hsp_framgent

/Users/am60/miniconda3/envs/biipy/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)
/Users/am60/miniconda3/envs/biipy/lib/python3.5/site-packages/Bio/SearchIO/BlastIO/blast_xml.py:336: BiopythonParserWarning: Renaming hit ID '2R' to a BLAST-generated ID 'gnl|BL_ORD_ID|2318' since the ID was already matched by your query '129'. Your BLAST database may contain duplicate entries.
  (hit.id, hit.blast_id, query_id), BiopythonParserWarning)
/Users/am60/miniconda3/envs/biipy/lib/python3.5/site-packages/Bio/SearchIO/BlastIO/blast_xml.py:336: BiopythonParserWarning: Renaming hit ID '2R' to a BLAST-generated ID 'gnl|BL_ORD_ID|2318' since the ID was already matched by your query '278'. Your BLAST database may contain duplicate entries.
  (hit.id, hit.blast_id, query_id), BiopythonParserWarning)
/Users/am60/mini

      Query: 129 
        Hit: 2L chromosome:AgamP3:2L:1:49364325:1
Query range: [0:343] (1)
  Hit range: [46338833:46339176] (1)
Quick stats: evalue 7e-180; bitscore 634.52
  Fragments: 1 (343 columns)
     Query - CATCAAAATGGGTAAGGAGAAGACTCATATTAACATCGTCGTCATCGGACACGTCGACT~~~GTGCC
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - CATCAAAATGGGTAAGGAGAAGACTCATATTAACATCGTCGTCATCGGACACGTCGACT~~~GTGCC
      Query: 129 
        Hit: supercont3.222 dna:supercontig supercontig:CpipJ2:supercont3.222...
Query range: [2:343] (1)
  Hit range: [571656:571997] (-1)
Quick stats: evalue 2.1e-140; bitscore 503.41
  Fragments: 1 (341 columns)
     Query - TCAAAATGGGTAAGGAGAAGACTCATATTAACATCGTCGTCATCGGACACGTCGACTCT~~~GTGCC
             |||| ||||||||||| ||||||||||||||||| ||||| || ||||||||||| || ~~~|||||
       Hit - TCAAGATGGGTAAGGAAAAGACTCATATTAACATTGTCGTGATTGGACACGTCGATTCC~~~GTGCC
      Query: 129 
        Hit: 2R chromosome:AgamP3:2R:1:61545105:1
Query rang

This is an example of 'bad' marker - multiple hits in three of four genomes. Other informative measures - % query coverage and % identity for each hit.

Warnings generated during parsing result from same chromosome names in different genomes: here it was 2R in Agam and Dmel. Need to take into account the additional field.

In [11]:
# look at one of the introns
meta = pd.read_csv(META_FILE)
meta[meta['intron'] == 'Yes']

,Unnamed: 0,aligned_len,aligned_species,chr,end,id_lineages,indels,informativeness,seqid,snvs,...,target_snvs,target_start,unid_species,exon,gene,intron,mRNA,repeat,utr,id lineage proportion
1,1,345,21,2L,2926735,18,96,0.857143,AgamP3.chr2L,65,...,52,2926537,"[{'AgamM1', 'AaraD1', 'AgamP3'}, {'AsteI2', 'A...",E014129A;E014130A,AGAP004734,Yes,AGAP004734-RA,None,None,0.857143
33,33,298,21,2L,10356642,20,139,0.952381,AgamP3.chr2L,38,...,24,10356481,"[{'AsteI2', 'AsteS1'}]",E016098A;E016099A,AGAP005134,Yes,AGAP005134-RA,None,None,0.952381
79,79,338,21,2L,26760891,20,157,0.952381,AgamP3.chr2L,47,...,40,26760778,"[{'AgamM1', 'AgamP3'}]",E020405C;E020406C;E020405B;E020406B;E020405A;E...,AGAP006107,Yes,AGAP006107-RC;AGAP006107-RB;AGAP006107-RA,None,None,0.952381
85,85,322,21,2L,27674840,20,178,0.952381,AgamP3.chr2L,35,...,20,27674693,"[{'AsteI2', 'AsteS1'}]",E020688A;E020689A;E020688B;E020689B,AGAP006176,Yes,AGAP006176-RA;AGAP006176-RB,None,None,0.952381
101,101,376,21,2L,34301269,20,159,0.952381,AgamP3.chr2L,51,...,32,34301104,"[{'AsteI2', 'AsteS1'}]",E022297D;E022298D;E022297C;E022298C;E022297B;E...,AGAP006590,Yes,AGAP006590-RD;AGAP006590-RC;AGAP006590-RB;AGAP...,None,None,0.952381
118,118,488,20,2L,42393512,20,137,1.000000,AgamP3.chr2L,65,...,47,42393238,[],E024360A;E024361A,AGAP007086,Yes,AGAP007086-RA,None,None,1.000000
119,119,591,20,2L,42393789,20,183,1.000000,AgamP3.chr2L,55,...,29,42393509,[],E024361A;E024362A,AGAP007086,Yes,AGAP007086-RA,None,None,1.000000
120,120,291,20,2L,42542495,11,17,0.550000,AgamP3.chr2L,30,...,12,42542260,"[{'AmelC1', 'AmerM1', 'AgamP3', 'AaraD1', 'Aqu...",E024408A;E024409A,AGAP007092,Yes,AGAP007092-RA,None,None,0.550000
128,128,419,20,2L,45853080,19,159,0.950000,AgamP3.chr2L,22,...,6,45852918,"[{'AsteI2', 'AsteS1'}]",E025379B;E025380B;E025379A;E025380A,AGAP007340,Yes,AGAP007340-RB;AGAP007340-RA,None,None,0.950000
133,133,283,21,2L,48612732,20,149,0.952381,AgamP3.chr2L,27,...,16,48612544,"[{'AsteI2', 'AsteS1'}]",E026619C;E026620C;E026619D;E026620D;E026619A;E...,AGAP007643,Yes,AGAP007643-RC;AGAP007643-RD;AGAP007643-RA;AGAP...,None,Yes,0.952381


In [12]:
# example intron
for hit in blast_qresult[33]:
    for hsp in hit:
        print(hsp)
# two hits in Aaeg correspond to flanking exons (according to query range), other species don't have that

      Query: 33 
        Hit: 2L chromosome:AgamP3:2L:1:49364325:1
Query range: [0:230] (1)
  Hit range: [10356412:10356642] (1)
Quick stats: evalue 2.9e-117; bitscore 425.85
  Fragments: 1 (230 columns)
     Query - GCCCGTGTGTACGGTCTGAAGAACATCCAGGCCGATGAGATGGTGGAGTTCTCCTCCGG~~~AGCTG
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - GCCCGTGTGTACGGTCTGAAGAACATCCAGGCCGATGAGATGGTGGAGTTCTCCTCCGG~~~AGCTG
      Query: 33 
        Hit: 2 dna:chromosome chromosome:AaegL5:2:1:474425716:1
Query range: [0:71] (1)
  Hit range: [204342875:204342946] (1)
Quick stats: evalue 1.6e-25; bitscore 121.15
  Fragments: 1 (71 columns)
     Query - GCCCGTGTGTACGGTCTGAAGAACATCCAGGCCGATGAGATGGTGGAGTTCTCCTCCGG~~~GTAAA
             |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||~~~|||||
       Hit - GCCCGTGTGTACGGTCTGAAGAACATCCAGGCCGATGAGATGGTGGAGTTCTCCTCCGG~~~GTAAA
      Query: 33 
        Hit: supercont3.30 dna:supercontig supercontig:CpipJ2:supercont3.30:

In [13]:
for hit in blast_qresult[33]:
    #print(hit)
    for hsp in hit:
            print(hsp.midline)
        
print 

AttributeError: 'HSP' object has no attribute 'midline'